# S1 Coursework

Yuanzhen Zhao (yz929)

In [1]:
import numpy as np
from scipy.stats import norm
from scipy.integrate import quad, dblquad
import matplotlib.pyplot as plt
from iminuit import Minuit
from iminuit.cost import ExtendedUnbinnedNLL
from dists import *
import timeit

from scipy.optimize import root_scalar
from scipy.interpolate import interp1d

In [2]:
# Parameters
mu, sigma, beta, m, f, lmda, mu_b, sigma_b = 3, 0.3, 1, 1.4, 0.6, 0.3, 0, 2.5

## Question b

In [16]:
# Check if normalisation is satisfied
print("CDF of g_s in range x=[0,5] is", quad(g_s, 0, 5, args=(beta, m, mu, sigma))[0])
print("CDF of h_s in range y=[0,10] is", quad(h_s, 0, 10, args=lmda)[0])
print("CDF of g_b in range x=[0,5] is", quad(g_b, 0, 5)[0])
print("CDF of h_b in range y=[0,10] is", quad(h_b, 0, 10, args=(mu_b, sigma_b))[0])
print("CDF of s_xy in range x=[0,5], y=[0,10] is", dblquad(s_xy, 0, 5, 0, 10, args=(beta, m, mu, sigma, lmda))[0])
print("CDF of b_xy in range x=[0,5], y=[0,10] is", dblquad(b_xy, 0, 5, 0, 10, args=(mu_b, sigma_b))[0])
print("CDF of f_xy in range x=[0,5], y=[0,10] is", dblquad(f_xy, 0, 5, 0, 10, args=(mu, sigma, beta, m, f, lmda, mu_b, sigma_b))[0])

CDF of g_s in range x=[0,5] is 1.0000000376260723
CDF of h_s in range y=[0,10] is 1.0
CDF of g_b in range x=[0,5] is 1.0000000000000002
CDF of h_b in range y=[0,10] is 1.0
CDF of s_xy in range x=[0,5], y=[0,10] is 1.0000000376260723
CDF of b_xy in range x=[0,5], y=[0,10] is 1.0000000000000002
CDF of f_xy in range x=[0,5], y=[0,10] is 1.0000000225756434


## Question c

In [ ]:
x = np.linspace(0, 5, 200)
y = np.linspace(0, 10, 200)

P_x = f * g_s(x, beta, m, mu, sigma) + (1-f) * g_b(x)
P_y = f * h_s(y, lmda) + (1-f) * h_b(y, mu_b, sigma_b)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 3))
ax[0].plot(x, g_s(x, beta, m, mu, sigma), label='$g_s(x)$')
ax[0].set_title('$g_s(x)$')
ax[0].legend()

ax[1].plot(x, g_b(x), label='$g_b(x)$')
ax[1].set_title('$g_b(x)$')
ax[1].legend()

ax[2].plot(x, P_x, label='$P_x$')
ax[2].set_title('$P_x = f \\cdot g_s + (1-f) \\cdot g_b$')
ax[2].legend()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 3))
ax[0].plot(y, h_s(y, lmda), label='$h_s(y)$')
ax[0].set_title('$h_s(y)$')
ax[0].legend()


ax[1].plot(y, h_b(y, mu_b, sigma_b), label='$h_b(y)$')
ax[1].set_title('$h_b(y)$')
ax[1].legend()

ax[2].plot(y, P_y, label='$P_y$')
ax[2].set_title('$P_y = f \\cdot h_s + (1-f) \\cdot h_b$')
ax[2].legend()

In [ ]:
X, Y = np.meshgrid(x, y)

fig, ax = plt.subplots(1, 3, figsize=(12, 3))

sxy = ax[0].contourf(X, Y, s_xy(Y, X, beta, m, mu, sigma, lmda), levels=100)
ax[0].set_title('$s_{xy}(x,y)$')
ax[0].set_xlabel('$x$')
ax[0].set_ylabel('$y$')
fig.colorbar(sxy, ax=ax[0])

bxy = ax[1].contourf(X, Y, b_xy(Y, X, mu_b, sigma_b), levels=100)
ax[1].set_title('$b_{xy}(x,y)$')
ax[1].set_xlabel('$x$')
ax[1].set_ylabel('$y$')
fig.colorbar(bxy, ax=ax[1])

fxy = ax[2].contourf(X, Y, f_xy(Y, X, mu, sigma, beta, m, f, lmda, mu_b, sigma_b), levels=100)
ax[2].set_title('$f_{xy} = f \\cdot s_{xy} + (1-f) \\cdot b_{xy}$')
ax[2].set_xlabel('$x$')
ax[2].set_ylabel('$y$')
fig.colorbar(fxy, ax=ax[2])

plt.show()


## Question d

P.P.F Method

In [ ]:
def generate_ppf(N, mu=3, sigma=0.3, beta=1, m=1.4, f=0.6, lmda=0.3, mu_b=0, sigma_b = 2.5, seed=50):
    np.random.seed(seed)
    g_s_model = crystalball(beta=beta, m=m, loc=mu, scale=sigma)
    h_s_model = truncexpon(b=10*lmda, loc=0, scale=1/lmda)
    g_b_model = uniform(loc=0, scale=5)
    h_b_model = truncnorm(a=(0-mu_b)/sigma_b, b=(10-mu_b)/sigma_b, loc=mu_b, scale=sigma_b)
    
    Ns = np.random.poisson(f * N)
    Nb = np.random.poisson((1-f) * N)
    
    g_s_uniform_samples = np.random.uniform(g_s_model.cdf(0), g_s_model.cdf(5), size=Ns)
    Xs = g_s_model.ppf(g_s_uniform_samples) # Make sure Xs are sampled in the range [0,5]
    
    g_b_uniform_samples = np.random.uniform(g_b_model.cdf(0), g_b_model.cdf(5), size=Nb)
    Xb = g_b_model.ppf(g_b_uniform_samples) # Make sure Xb are sampled in the range [0,5]
    
    Ys = h_s_model.rvs(size=Ns)
    Yb = h_b_model.rvs(size=Nb)
    
    sevs = np.reshape((Xs, Ys), (Ns, 2))
    bevs = np.reshape((Xb, Yb), (Nb, 2))
    
    return np.concatenate((sevs, bevs))

In [19]:
dataset = generate_ppf(100000)

def density(xy, mu=3, sigma=0.3, beta=1, m=1.4, f=0.6, lmda=0.3, mu_b=0, sigma_b=2.5, N=100000):
    x, y = xy
    
    if f > 0 and f < 1:
        Ns = np.random.poisson(f * N)
        Nb = np.random.poisson((1-f) * N)
    else:
        Ns = f * N
        Nb = (1-f) * N
    return Ns+Nb, Ns*g_s(x, beta, m, mu, sigma)*h_s(y, lmda) + Nb*g_b(x) * h_b(y, mu_b, sigma_b)

In [21]:
nll = ExtendedUnbinnedNLL((dataset[:, 0], dataset[:, 1]), density, log=False, verbose=False)
mi = Minuit(nll, mu=3, sigma=0.3, beta=1, m=1.4, f=0.6, lmda=0.3, mu_b=0, sigma_b=2.5, N=100000)
mi.limits["mu"]=(0, 5)
mi.limits["sigma"]=(0, 0.5)
mi.limits["beta"]=(0, 2)
mi.limits["m"]=(1, 2)
mi.limits["f"]=(0.3, 0.7)
mi.limits["lmda"]=(0, 1)
mi.limits["mu_b"]=(-3,3)
mi.limits["sigma_b"]=(0,5)
mi.limits["N"]=(1, None)

mi.migrad()
mi.hesse()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = -1.362e+06                 │             Nfcn = 2291              │
│ EDM = nan (Goal: 0.0002)         │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│         INVALID Minimum          │   ABOVE EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │     Covariance FORCED pos. def.      │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬─────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name    │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼─────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ mu      │  4.2768   │    nan    │            │            │    0    │    5    │       │
│ 1 │ sigma   │ 4.5812e-1 │    nan    │            │            │    0    │   0.5   │       │
│ 2 │ beta    │     1     │    nan    │            │            │    0    │    2    │       │
│ 3 │ m       │  1.3306   │    nan    │            │            │    1    │    2    │       │
│ 4 │ f       │ 3.2147e-1 │    nan    │            │            │   0.3   │   0.7   │       │
│ 5 │ lmda    │ 2.8275e-1 │    nan    │            │            │    0    │    1    │       │
│ 6 │ mu_b    │-2.8585e-8 │    nan    │            │            │   -3    │    3    │       │
│ 7 │ sigma_b │    2.5    │    nan    │            │            │    0    │    5    │       │
│ 8 │ N       │ 1.2682e5  │    nan    │            │            │    1    │         │       │
└───┴─────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────────┬─────────────────────────────────────────────────────────────────────────┐
│         │      mu   sigma    beta       m       f    lmda    mu_b sigma_b       N │
├─────────┼─────────────────────────────────────────────────────────────────────────┤
│      mu │     nan     nan     nan     nan     nan     nan     nan     nan     nan │
│   sigma │     nan     nan     nan     nan     nan     nan     nan     nan     nan │
│    beta │     nan     nan     nan     nan     nan     nan     nan     nan     nan │
│       m │     nan     nan     nan     nan     nan     nan     nan     nan     nan │
│       f │     nan     nan     nan     nan     nan     nan     nan     nan     nan │
│    lmda │     nan     nan     nan     nan     nan     nan     nan     nan     nan │
│    mu_b │     nan     nan     nan     nan     nan     nan     nan     nan     nan │
│ sigma_b │     nan     nan     nan     nan     nan     nan     nan     nan     nan │
│       N │     nan     nan     nan     nan     nan     nan     nan     nan     nan │
└─────────┴─────────────────────────────────────────────────────────────────────────┘

Accept-Reject method

In [10]:
from scipy.optimize import brute, minimize


In [ ]:
def accept_reject_batch(func, xrange, yrange, size, fmax=None, seed=None):
    if seed is not None:
        np.random.seed(seed)

    # Step 1: Estimate fmax if not provided
    if fmax is None:
        print("Estimating fmax... ", end="")
        f_to_min = lambda xy: -func(xy[0], xy[1])
        xy0 = brute(f_to_min, [xrange, yrange], finish=None)
        xy0 = minimize(f_to_min, xy0, bounds=[xrange, yrange]).x
        fmax = func(xy0[0], xy0[1])
        print(f"DONE. fmax = {fmax}")

    # Step 2: Generate a large dataset
    n_points = int(size * 20)  # Oversampling to ensure enough accepted samples
    x = np.random.uniform(*xrange, n_points)
    y = np.random.uniform(*yrange, n_points)
    f_vals = func(x, y)

    # Ensure function values are non-negative
    if np.any(f_vals < 0):
        raise ValueError("PDF values must be non-negative.")

    # Step 3: Generate uniform random values for accept-reject
    random_vals = np.random.uniform(0, fmax, n_points)

    # Step 4: Accept points where random value <= f(x, y)
    accept_mask = random_vals <= f_vals
    accepted_x = x[accept_mask]
    accepted_y = y[accept_mask]

    # Step 5: Check if we have enough points
    if len(accepted_x) < size:
        raise ValueError(
            "Not enough accepted points generated. Increase n_points or fmax."
        )

    # Step 6: Return the required number of samples
    accepted_samples = np.column_stack((accepted_x[:size], accepted_y[:size]))
    return accepted_samples

In [17]:
samples = accept_reject_batch(
    func=lambda x, y: f_xy(y, x, mu=3, sigma=0.3, beta=1, m=1.4, f=0.6, lmda=0.3, mu_b=0, sigma_b = 2.5),
    xrange=(0, 5),
    yrange=(0, 10),
    size=100000,
    seed=15
)

Estimating fmax... DONE. fmax = 0.21804724534435388


In [18]:
nll = ExtendedUnbinnedNLL((samples[:, 0], samples[:, 1]), density, log=False, verbose=False)
mi = Minuit(nll, mu=3, sigma=0.3, beta=1, m=1.4, f=0.6, lmda=0.3, mu_b=0, sigma_b=2.5, N=100000)
mi.limits["mu"]=(0, 5)
mi.limits["sigma"]=(0, 0.5)
mi.limits["beta"]=(0, 2)
mi.limits["m"]=(1, 2)
mi.limits["f"]=(0.3, 0.7)
mi.limits["lmda"]=(0, 1)
mi.limits["mu_b"]=(-3,3)
mi.limits["sigma_b"]=(0,5)
mi.limits["N"]=(0, None)

mi.migrad()
mi.hesse()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = -1.461e+06                 │             Nfcn = 3313              │
│ EDM = 7.59e+04 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│         INVALID Minimum          │   ABOVE EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │     Covariance FORCED pos. def.      │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬─────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name    │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼─────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ mu      │  2.9992   │  0.0010   │            │            │    0    │    5    │       │
│ 1 │ sigma   │ 301.4e-3  │  0.8e-3   │            │            │    0    │   0.5   │       │
│ 2 │ beta    │   1.005   │   0.011   │            │            │    0    │    2    │       │
│ 3 │ m       │   1.39    │   0.18    │            │            │    1    │    2    │       │
│ 4 │ f       │   0.605   │   0.008   │            │            │   0.3   │   0.7   │       │
│ 5 │ lmda    │  0.2991   │  0.0016   │            │            │    0    │    1    │       │
│ 6 │ mu_b    │   0.058   │   0.012   │            │            │   -3    │    3    │       │
│ 7 │ sigma_b │   2.495   │   0.006   │            │            │    0    │    5    │       │
│ 8 │ N       │ 100.004e3 │  0.017e3  │            │            │    0    │         │       │
└───┴─────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│         │          mu       sigma        beta           m           f        lmda        mu_b     sigma_b           N │
├─────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│      mu │     9.8e-07     -0.2e-6     -5.9e-6     97.3e-6      4.0e-6      0.7e-6     -0.8e-6      0.4e-6  -8.9358e-3 │
│   sigma │     -0.2e-6    6.32e-07      3.2e-6    -51.4e-6     -2.1e-6     -0.4e-6      0.4e-6     -0.2e-6   4.7291e-3 │
│    beta │     -5.9e-6      3.2e-6     0.00013    -2.03e-3    -0.08e-3    -14.9e-6     0.02e-3   -0.008e-3   185.43e-3 │
│       m │     97.3e-6    -51.4e-6    -2.03e-3      0.0347     1.41e-3    254.4e-6    -0.29e-3    0.147e-3      -3.164 │
│       f │      4.0e-6     -2.1e-6    -0.08e-3     1.41e-3    5.89e-05     10.4e-6    -0.01e-3    0.006e-3  -129.30e-3 │
│    lmda │      0.7e-6     -0.4e-6    -14.9e-6    254.4e-6     10.4e-6    2.51e-06     -1.6e-6      1.3e-6 -23.2968e-3 │
│    mu_b │     -0.8e-6      0.4e-6     0.02e-3    -0.29e-3    -0.01e-3     -1.6e-6    0.000149   -0.010e-3    26.74e-3 │
│ sigma_b │      0.4e-6     -0.2e-6   -0.008e-3    0.147e-3    0.006e-3      1.3e-6   -0.010e-3    3.15e-05  -13.406e-3 │
│       N │  -8.9358e-3   4.7291e-3   185.43e-3      -3.164  -129.30e-3 -23.2968e-3    26.74e-3  -13.406e-3         290 │
└─────────┴─────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [ ]:
time1 = timeit.timeit(lambda: np.random.normal(size=100000), number=100)
time2 = timeit.timeit(lambda: accept_reject_batch(
    func=lambda x, y: f_xy(y, x, mu=3, sigma=0.3, beta=1, m=1.4, f=0.6, lmda=0.3, mu_b=0, sigma_b = 2.5),
    xrange=(0, 5),
    yrange=(0, 10),
    size=100000,
    seed=42
), number=100)
time3 = timeit.timeit(lambda: fit(dataset), number=100)

print("Time for normal random numbers:", time1)
print("Time for generating dataset:", time2)
print("Time for fitting dataset:", time3)

## Question e

Now run a simulation study using parametric bootstrapping (with an ensemble
of at least 250 samples) from the true probability distribution. You should trial
sample sizes of 500, 1000, 2500, 5000 and 10000, including a Poisson variation on
the sample size. Determine whether you see any bias on the 𝜆 parameter,
describing the decay constant of the signal in 𝑌, as a function of the sample size.
Also determine the expected uncertainty on 𝜆 as a function of the sample size.

现在进行一个模拟研究，使用参数自助法（从真实概率分布中生成至少250个样本的集合）。您应该尝试以下样本量：500、1000、2500、5000和10000，并包括样本量上的泊松变化。确定是否可以观察到𝜆参数（描述信号在𝑌中的衰减常数）的偏差随样本量的变化情况。同时，确定𝜆的预期不确定性如何随样本量的变化而变化

In [23]:
sample_sizes = [50, 1000, 2500, 5000, 10000]

mi.values["lmda"]
#mi.errors["lmda"]

0.29907292607760827

In [ ]:
from resample import bootstrap
for sample_size in sample_sizes:
    bstrp = np.array( [ b for b in bootstrap.resample(samples, size=sample_size) ] )